<a href="https://colab.research.google.com/github/AdrianaUP/News-classifier_AGNews/blob/main/agnews_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers scikit-learn matplotlib seaborn pandas


In [4]:
from datasets import load_dataset

# Cargar dataset
dataset = load_dataset("ag_news")

# Dividir en 70% train, 15% val, 15% test
base_split = dataset["train"].train_test_split(test_size=0.3, seed=42)
train_val = base_split["train"].train_test_split(test_size=0.1765, seed=42)  # 15% de total

train_dataset = train_val["train"]
val_dataset = train_val["test"]
test_dataset = base_split["test"]

print(f"Train: {len(train_dataset)}")
print(f"Val: {len(val_dataset)}")
print(f"Test: {len(test_dataset)}")


Train: 69174
Val: 14826
Test: 36000


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"f1": f1_score(labels, preds, average="weighted")}

def train_model(model_name, train_dataset, val_dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

    def tokenize(batch):
        return tokenizer(batch["text"], truncation=True, padding=True)

    train_dataset = train_dataset.map(tokenize, batched=True)
    val_dataset = val_dataset.map(tokenize, batched=True)

    training_args = TrainingArguments(
        output_dir=f"./results/{model_name}",
        evaluation_strategy="epoch",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        logging_dir="./logs",
        logging_steps=10,
        save_strategy="no"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    return trainer
